# Introduction

Today we will demonstrate how you can generate advertizing content for your inventory to boost sales. We'll do this taking advantage of Azure Cosmos DB for Mongo DB vCore's [vector similarity search](https://learn.microsoft.com/azure/cosmos-db/mongodb/vcore/vector-search) functionality. We will use OpenAI embeddings to generate vectors for inventory description which expected to vastly enhance its semantics. The vectors are then stored and indexed in the Mongo vCore Database. During the content generation for the advertisement time we will also vectorize the advertisement topic and find matching inventory itmes. We will then use retrival augmented generation (RAG) by sending to top matches to OpenAI to generate a catchy advertisement.

# Scenario

1. Shoe Retailer who wants to sell more shoes 
2. Wants to run advertisement to capitalize on recent trends
2. Wants to use generate advertisement content using the inventory items that matches the trend

## Azure OpenAI <a class="anchor" id="azureopenai"></a>

Finally, let's setup our Azure OpenAI resource Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form at https://aka.ms/oai/access. Once you have access, complete the following steps:

- Create an Azure OpenAI resource following this quickstart: https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal
- Deploy a `completions` and `embeddings` model 
    - For more information on `completions`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/completions
    - For more information on `embeddings`, go here: https://learn.microsoft.com/azure/ai-services/openai/how-to/embeddings
- Copy the endpoint, key, deployment names for (embeddings model, completions model) into the config.json file.

## Create an Azure Cosmos DB for MongoDB vCore resource<a class="anchor" id="cosmosdb"></a>
Let's start by creating an Azure Cosmos DB for MongoDB vCore Resource following this quick start guide: https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/quickstart-portal

Then copy the connection details (server, user, pwd) into the config.json file.

# Preliminaries <a class="anchor" id="preliminaries"></a>
First, let's start by installing the packages that we'll need later. 

In [ ]:
! pip install numpy
! pip install openai==1.2.3
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity
! pip install gradio

Please use the example.env as a template to provide the necessary keys and endpoints in your own .env file.
Make sure to modify the env_name accordingly. 

In [187]:
import json
import datetime
import time

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential

import openai
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

from dotenv import dotenv_values
from openai import AzureOpenAI

# specify the name of the .env file name 
env_name = "adgen.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

COSMOS_MONGO_USER = config['cosmos_db_mongo_user']
COSMOS_MONGO_PWD = config['cosmos_db_mongo_pwd']
COSMOS_MONGO_SERVER = config['cosmos_db_mongo_server']

openai.api_type = config['openai_api_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']
embeddings_deployment = config['openai_embeddings_deployment']
completions_deployment = config['openai_completions_deployment']

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

# Load data and create embeddings <a class="anchor" id="loaddata"></a>
Here we'll load a sample dataset containing descriptions of Azure services. Then we'll user Azure OpenAI to create vector embeddings from this data.

In [200]:
#@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = client.embeddings.create(
        input=text, model="text-embedding-ada-002")
    print(response)
    embeddings = response.data[0].embedding
    time.sleep(0.5) # rest period to avoid rate limiting on AOAI for free tier
    return embeddings

generate_embeddings("example")

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.005542438477277756, -0.010508125647902489, -0.0027870447374880314, -0.009185818023979664, -0.008897442370653152, 0.019243797287344933, 0.00030398130184039474, -0.006942115258425474, -0.005735861137509346, -0.026643091812729836, 0.01958140730857849, 0.032213665544986725, 0.00459642568603158, -0.0096078310161829, -0.0023562395945191383, 0.009657066315412521, 0.04163862392306328, -0.003332145046442747, 0.017991824075579643, -0.018779581412672997, -0.013877197168767452, 0.01797775737941265, -0.01319494191557169, 0.001184274209663272, -0.015108068473637104, -0.007722839247435331, 0.024139149114489555, -0.028668755665421486, 0.009347589686512947, -0.003131688805297017, 0.01901872269809246, -0.002774735912680626, -0.01915939338505268, -0.014742323197424412, 0.006453283596783876, -0.005939834285527468, 0.010796501301229, -0.013659156858921051, -0.0012264754623174667, -0.0028626553248614073, 0.008039348758757114, -0.01747134141623974, 0.0122

[-0.005542438477277756,
 -0.010508125647902489,
 -0.0027870447374880314,
 -0.009185818023979664,
 -0.008897442370653152,
 0.019243797287344933,
 0.00030398130184039474,
 -0.006942115258425474,
 -0.005735861137509346,
 -0.026643091812729836,
 0.01958140730857849,
 0.032213665544986725,
 0.00459642568603158,
 -0.0096078310161829,
 -0.0023562395945191383,
 0.009657066315412521,
 0.04163862392306328,
 -0.003332145046442747,
 0.017991824075579643,
 -0.018779581412672997,
 -0.013877197168767452,
 0.01797775737941265,
 -0.01319494191557169,
 0.001184274209663272,
 -0.015108068473637104,
 -0.007722839247435331,
 0.024139149114489555,
 -0.028668755665421486,
 0.009347589686512947,
 -0.003131688805297017,
 0.01901872269809246,
 -0.002774735912680626,
 -0.01915939338505268,
 -0.014742323197424412,
 0.006453283596783876,
 -0.005939834285527468,
 0.010796501301229,
 -0.013659156858921051,
 -0.0012264754623174667,
 -0.0028626553248614073,
 0.008039348758757114,
 -0.01747134141623974,
 0.012294648215

In [ ]:
# Generate embeddings for title and content fields

data_file = open(file="./path_to_output_file_img_download.json", mode="r") 
data = json.load(data_file)
data_file.close()

n = 0
for item in data:
    n+=1
    product_string = f"name: {item['name']}" + \
        (f", color: {item['color']}" if item['color'] else "") + \
        (f", material: {item['material']}" if item['material'] else "") + \
        (f", occasion: {item['occasion']}" if item['occasion'] else "") + \
        (f", brand: {item['brand']}" if item['brand'] else "")
    title_embeddings = generate_embeddings(product_string)
    item['contentVector'] = title_embeddings
    print("Creating embeddings for item:", n, "/" ,len(data), end='\r')
# Save embeddings to sample_text_w_embeddings.json file
with open("./output_embeddings.json", "w") as f:
    json.dump(data, f, indent=1)

In [ ]:
import csv
import json
import time
import requests
from openai import OpenAI

def csv_to_json_complete(input_csv_file, output_json_file):
    # Open the CSV file for reading

    daliClient = OpenAI(
        api_key=config['dali_api_key']
    )

    with open(input_csv_file, 'r', newline='', encoding='utf-8') as csvfile:
        # Create a CSV reader object
        reader = csv.DictReader(csvfile)
        
        # Initialize an empty list to hold all the JSON objects
        json_list = []
        
        # Initialize an ID counter starting at 1
        auto_id = 1
        
        # Iterate over each row in the CSV file
        for row in reader:
            # Add the 'id' key with an auto-incremented value

            if auto_id > 300:
                break

            row['id'] = auto_id
            # Convert string representations of boolean and None types to actual boolean and None types
            for key, value in row.items():
                if value == 'true': 
                    row[key] = True
                elif value == 'false': 
                    row[key] = False
                elif value == '':
                    row[key] = None  # Convert empty strings to None

            product_string = f"name: {row['name']}" + \
                    (f", color: {row['color']}" if row['color'] else "") + \
                    (f", material: {row['material']}" if row['material'] else "") + \
                    (f", occasion: {row['occasion']}" if row['occasion'] else "") + \
                    (f", brand: {row['brand']}" if row['brand'] else "")

            try: 
                response = daliClient.images.generate(
                    model="dall-e-3",
                    prompt= "Generate a photo realistic image of a shoe with the following description " + product_string + ". The image will be used to show inventory item online.",
                    size="1024x1024",
                    quality="standard",
                    n=1,
                    )

                image_url = response.data[0].url
                print(image_url)

                row['img_url'] = image_url
                # Append the modified row to the list of JSON objects
                json_list.append(row)
                # Increment the ID counter

                response = requests.get(image_url)
                local_file_name = "./images/"+str(auto_id)+".png"
                # Check if the request was successful
                if response.status_code == 200:
                    # Open a local file in binary write mode
                    with open(local_file_name, 'wb') as file:
                        # Write the content of the response to the file
                        file.write(response.content)
                    print(f"Image downloaded successfully: {local_file_name}")
                else:
                    print(f"Error: Failed to retrieve image from {image_url}")
                auto_id += 1
            except Exception as e:  # Catch any exception
                print(f"An exception occurred for desc{product_string}:", e)
                        
            # Sleep for 30 seconds after every 5 rows
            if auto_id % 5 == 0:
                print(f"Processed 5 rows, sleeping for 30 seconds at row {auto_id}...")
                time.sleep(30)

    # Open the JSON file for writing
    with open(output_json_file, 'w', encoding='utf-8') as jsonfile:
        # Write the list of JSON objects to the file
        json.dump(json_list, jsonfile, indent=4)

# Example usage
input_csv_file = 'shoes.csv'  # Replace with your actual input CSV file path
output_json_file = 'path_to_output_file_img_download.json'  # Replace with your actual output JSON file path
# csv_to_json_complete(input_csv_file, output_json_file)

# Connect and setup Cosmos DB for MongoDB vCore

## Set up the connection

In [188]:
import pymongo

mongo_conn = config['mongo_vcore_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

##  Set up the DB and collection

In [189]:
DATABASE_NAME = "AdgenDatabase13"
COLLECTION_NAME = "AdgenCollection13"

mongo_client.drop_database(DATABASE_NAME)
db = mongo_client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

if COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    db.create_collection(COLLECTION_NAME)
    print("Created collection '{}'.\n".format(COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COLLECTION_NAME))

Created collection 'AdgenCollection12'.



## Create the vector index

In [190]:
db.command({
  'createIndexes': COLLECTION_NAME,
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
});

## Upload data to the collection
A simple `insert_many()` to insert our data in JSON format into the newly created DB and collection.

In [191]:
data_file = open(file="./output_embeddings.json", mode="r") 
data = json.load(data_file)
data_file.close()

collection.insert_many(data)

InsertManyResult([ObjectId('6551d893da845cf81d5d53e8'), ObjectId('6551d893da845cf81d5d53e9'), ObjectId('6551d893da845cf81d5d53ea'), ObjectId('6551d893da845cf81d5d53eb'), ObjectId('6551d893da845cf81d5d53ec'), ObjectId('6551d893da845cf81d5d53ed'), ObjectId('6551d893da845cf81d5d53ee'), ObjectId('6551d893da845cf81d5d53ef'), ObjectId('6551d893da845cf81d5d53f0'), ObjectId('6551d893da845cf81d5d53f1'), ObjectId('6551d893da845cf81d5d53f2'), ObjectId('6551d893da845cf81d5d53f3'), ObjectId('6551d893da845cf81d5d53f4'), ObjectId('6551d893da845cf81d5d53f5'), ObjectId('6551d893da845cf81d5d53f6'), ObjectId('6551d893da845cf81d5d53f7'), ObjectId('6551d893da845cf81d5d53f8'), ObjectId('6551d893da845cf81d5d53f9'), ObjectId('6551d893da845cf81d5d53fa'), ObjectId('6551d893da845cf81d5d53fb'), ObjectId('6551d893da845cf81d5d53fc'), ObjectId('6551d893da845cf81d5d53fd'), ObjectId('6551d893da845cf81d5d53fe'), ObjectId('6551d893da845cf81d5d53ff'), ObjectId('6551d893da845cf81d5d5400'), ObjectId('6551d893da845cf81d5d54

# Vector Search in Cosmos DB for MongoDB vCore

In [192]:
# Simple function to assist with vector search
def vector_search(query, num_results=3):
    query_embedding = generate_embeddings(query)
    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "numLists": 1,
                    "path": "contentVector",
                    "k": num_results
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

## Perform vector search query

In [193]:
query = "Shoes for Seattle sweater weather"
results = vector_search(query, 5)
for result in results: 
    img_url = "https://ignitedemotorage.z5.web.core.windows.net/images/" + str(result['document']['id'])+ ".png"
    print(f"Similarity Score: {result['similarityScore']}")  
    print(f"Title: {result['document']['name']}")  
    print(f"Content: {result['document']['price']}")  
    print(f"Category: {result['document']['material']}\n") 
    print(f"Image: {img_url}\n") 

CreateEmbeddingResponse(data=[Embedding(embedding=[0.011704880744218826, -0.020107725635170937, -0.0054541402496397495, -0.019234035164117813, -0.012334451079368591, 0.014364496804773808, -0.015520851127803326, -0.011454337276518345, 0.014467284083366394, -0.034459374845027924, -0.0010776584967970848, -0.004124332219362259, 0.004654328338801861, -0.01027870923280716, -0.007381398230791092, -0.029885347932577133, 0.02772681973874569, 0.011981121264398098, 0.021469654515385628, -0.032609205693006516, -0.015880607068538666, 0.013670683838427067, 0.029165837913751602, -0.01287408359348774, -0.005813895259052515, -0.0216752290725708, 0.01977366767823696, -0.03101600706577301, -0.0031060974579304457, -0.013208141550421715, 0.028189361095428467, -0.012340875342488289, -0.028523419052362442, -0.015199641697108746, -0.007676911074668169, -0.015366670675575733, -0.0023271641694009304, 0.007175824139267206, -0.011145975440740585, -0.017872106283903122, 0.008993870578706264, -0.014081832021474838,

# Generating Ad content with GPT-4

Finally, we'll create a helper function to feed prompts into the `Completions` model. Then we'll create interactive loop where you can pose questions to the model and receive information grounded in your data.

In [198]:
from openai import OpenAI

def Generate_ad_image(subject):
    daliClient = OpenAI(
        api_key=config['dali_api_key']
    )
    response = daliClient.images.generate(
        model="dall-e-3",
        prompt= subject,
        size="1024x1024",
        quality="standard",
        n=1,
        )

    return response.data[0].url

def render_html_page(ad_topic):

    # Find the matching shoes from the inventory
    results = vector_search(ad_topic, 4)

    ad_header = generate_completion("Generate a catchy, witty, and short sentence (less than 100 characters) for an advertisement for selling shoes for " + ad_topic)

    # image_url = "https://ignitedemotorage.z5.web.core.windows.net/images/0.png";
    image_prompt = f'''
        Generate a photorealistic image of an ad campaign for selling {ad_topic}. 
        The image should be clean, with the item being sold in the foreground with an easily identifiable landmark of the city in the background.
        The image should also try to depict the weather of the location for the time of the year mentioned.
        The image should not have any generated text overlay.
    '''
    image_url = Generate_ad_image(image_prompt)


    with open('ad-start.html', 'r', encoding='utf-8') as html_file:
        html_content = html_file.read()

    html_content += f'''<header>
            <h2>{ad_header}</h1>
        </header>'''    

    html_content += f'''
            <section class="ad">
            <img src="{image_url}" alt="Base Ad Image" class="ad-image">
        </section>'''

    for result in results: 
        img_url = "https://ignitedemotorage.z5.web.core.windows.net/images/" + str(result['document']['id'])+ ".png"

        html_content += f''' 
        <section class="product">
            <img src="{img_url}" alt="{result['document']['name']}" class="product-image">
            <div class="product-details">
                <h3 class="product-title">{result['document']['name']}</h2>
                <p class="product-price">{str(result['document']['price'])}</p>
                <p class="product-description">{result['document']['name']}</p>
                <a href="#" class="buy-now-button">Buy Now</a>
            </div>
        </section>
        '''

    html_content += '''</article>
                    </body>
                    </html>'''

    print(html_content)
    return html_content

# Putting it all together

In [199]:
import gradio as gr

css = """
    button { background-color: purple; color: read; }
    <style>
    </style>
"""

with gr.Blocks(css=css, theme=gr.themes.Default(spacing_size=gr.themes.sizes.spacing_sm, radius_size="none")) as demo:
    subject = gr.Textbox(placeholder="subject", label="Ad keywords")
    btn = gr.Button("Generate Ad")
    output_html = gr.HTML(label="Generated Ad HTML")

    btn.click(render_html_page, [subject], output_html)

    btn = gr.Button("Copy HTML")

if __name__ == "__main__":
    demo.launch()   

Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.


CreateEmbeddingResponse(data=[Embedding(embedding=[0.011704880744218826, -0.020107725635170937, -0.0054541402496397495, -0.019234035164117813, -0.012334451079368591, 0.014364496804773808, -0.015520851127803326, -0.011454337276518345, 0.014467284083366394, -0.034459374845027924, -0.0010776584967970848, -0.004124332219362259, 0.004654328338801861, -0.01027870923280716, -0.007381398230791092, -0.029885347932577133, 0.02772681973874569, 0.011981121264398098, 0.021469654515385628, -0.032609205693006516, -0.015880607068538666, 0.013670683838427067, 0.029165837913751602, -0.01287408359348774, -0.005813895259052515, -0.0216752290725708, 0.01977366767823696, -0.03101600706577301, -0.0031060974579304457, -0.013208141550421715, 0.028189361095428467, -0.012340875342488289, -0.028523419052362442, -0.015199641697108746, -0.007676911074668169, -0.015366670675575733, -0.0023271641694009304, 0.007175824139267206, -0.011145975440740585, -0.017872106283903122, 0.008993870578706264, -0.014081832021474838,